# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [4]:
my_columns = ['Ticker', 'Stock Price', '1 Year Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        '1 Year Return': [yr1Return],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  1 Year Return Number of Shares to Buy
0        A     110.1800      -0.095828                     N/A
1      AAL      13.2099       0.027537                     N/A
2      AAP      59.0500      -0.633660                     N/A
3     AAPL     175.4900       0.148946                     N/A
4     ABBV     153.6000       0.074121                     N/A
..     ...          ...            ...                     ...
498    YUM     125.5600       0.146764                     N/A
499    ZBH     118.2400       0.113878                     N/A
500   ZBRA     231.8300      -0.135964                     N/A
501   ZION      34.4200      -0.366816                     N/A
502    ZTS     177.7100       0.214238                     N/A

[503 rows x 4 columns]


In [6]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,110.1800,-0.095828,N/A
1,AAL,13.2099,0.027537,N/A
2,AAP,59.0500,-0.633660,N/A
3,AAPL,175.4900,0.148946,N/A
4,ABBV,153.6000,0.074121,N/A
...,...,...,...,...
498,YUM,125.5600,0.146764,N/A
499,ZBH,118.2400,0.113878,N/A
500,ZBRA,231.8300,-0.135964,N/A
501,ZION,34.4200,-0.366816,N/A


In [7]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
80,CAT,274.3,NaN,N/A


Filling the `NULL` cells of the `1 Year Return` column with mean value

In [8]:
final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)

In [9]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


In [10]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,110.1800,-0.095828,N/A
1,AAL,13.2099,0.027537,N/A
2,AAP,59.0500,-0.633660,N/A
3,AAPL,175.4900,0.148946,N/A
4,ABBV,153.6000,0.074121,N/A
...,...,...,...,...
498,YUM,125.5600,0.146764,N/A
499,ZBH,118.2400,0.113878,N/A
500,ZBRA,231.8300,-0.135964,N/A
501,ZION,34.4200,-0.366816,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [11]:
final_dataframe.sort_values('1 Year Return', ascending=False, inplace=True)

In [12]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [13]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,415.1252,2.362710,N/A
1,GE,113.8000,1.250962,N/A
2,META,299.7331,1.098236,N/A
3,FICO,897.9550,1.056166,N/A
4,RCL,95.4100,1.047021,N/A
5,PHM,74.2100,0.964791,N/A
6,ADBE,518.8450,0.866439,N/A
7,ACGL,80.9200,0.851127,N/A
8,LLY,550.4600,0.832406,N/A
9,AXON,196.4200,0.816436,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [14]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [16]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,415.1252,2.362710,481
1,GE,113.8000,1.250962,1757
2,META,299.7331,1.098236,667
3,FICO,897.9550,1.056166,222
4,RCL,95.4100,1.047021,2096
5,PHM,74.2100,0.964791,2695
6,ADBE,518.8450,0.866439,385
7,ACGL,80.9200,0.851127,2471
8,LLY,550.4600,0.832406,363
9,AXON,196.4200,0.816436,1018


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is that low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Here `hqm` stands for `high-quality momentum`.

In [17]:
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    '1 Year Return',
    '1 Year Return Percentile',
    '6 Months Return',
    '6 Months Return Percentile',
    '3 Months Return',
    '3 Months Return Percentile',
    '1 Month Return',
    '1 Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [18]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='6mo')
        mnth6Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth6Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='3mo')
        mnth3Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth3Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='1mo')
        mnth1Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth1Return = np.NaN
    
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        'Number of Shares to Buy': ['N/A'],
        '1 Year Return': [yr1Return],
        '1 Year Return Percentile': ['N/A'],
        '6 Months Return': [mnth6Return],
        '6 Months Return Percentile': ['N/A'],
        '3 Months Return': [mnth3Return],
        '3 Months Return Percentile': ['N/A'],
        '1 Month Return': [mnth1Return],
        '1 Month Return Percentile': ['N/A'],
        'HQM Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    hqm_dataframe = pd.concat([hqm_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(hqm_dataframe['Ticker'])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
498     YUM
499     ZBH
500    ZBRA
501    ZION
502     ZTS
Name: Ticker, Length: 503, dtype: object


In [19]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,110.1300,N/A,-0.095828,N/A,-19.334310,N/A,-6.752049,N/A,-7.344778,N/A,N/A
1,AAL,13.2150,N/A,0.027537,N/A,-8.037576,N/A,-18.926376,N/A,-12.829813,N/A,N/A
2,AAP,58.9800,N/A,-0.633660,N/A,-49.970784,N/A,-13.869108,N/A,-13.519058,N/A,N/A
3,AAPL,175.3911,N/A,0.148946,N/A,10.409182,N/A,-4.535320,N/A,-0.264443,N/A,N/A
4,ABBV,153.5945,N/A,0.074121,N/A,-0.028126,N/A,13.471623,N/A,2.287223,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,NaN,N/A,-2.497088,N/A,-8.336996,N/A,-2.686341,N/A,N/A
499,ZBH,118.4250,N/A,NaN,N/A,-7.328428,N/A,-17.121560,N/A,-1.599498,N/A,N/A
500,ZBRA,230.5000,N/A,NaN,N/A,-21.713139,N/A,-14.906971,N/A,-13.841438,N/A,N/A
501,ZION,34.4400,N/A,NaN,N/A,6.658401,N/A,23.220028,N/A,-0.260643,N/A,N/A


In [20]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
80,CAT,274.6459,N/A,NaN,N/A,23.379071,N/A,14.297673,N/A,0.695107,N/A,N/A
207,GNRC,109.8400,N/A,NaN,N/A,-4.262187,N/A,-16.472780,N/A,-5.444136,N/A,N/A
208,GOOG,132.7300,N/A,NaN,N/A,25.420446,N/A,9.471378,N/A,2.958972,N/A,N/A
209,GOOGL,131.7750,N/A,NaN,N/A,25.590933,N/A,9.307342,N/A,2.648601,N/A,N/A
210,GPC,146.9900,N/A,NaN,N/A,-11.409109,N/A,-8.967607,N/A,-4.477514,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,NaN,N/A,-2.497088,N/A,-8.336996,N/A,-2.686341,N/A,N/A
499,ZBH,118.4250,N/A,NaN,N/A,-7.328428,N/A,-17.121560,N/A,-1.599498,N/A,N/A
500,ZBRA,230.5000,N/A,NaN,N/A,-21.713139,N/A,-14.906971,N/A,-13.841438,N/A,N/A
501,ZION,34.4400,N/A,NaN,N/A,6.658401,N/A,23.220028,N/A,-0.260643,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [21]:
hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)

In [22]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [23]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,110.1300,N/A,-0.095828,N/A,-19.334310,N/A,-6.752049,N/A,-7.344778,N/A,N/A
1,AAL,13.2150,N/A,0.027537,N/A,-8.037576,N/A,-18.926376,N/A,-12.829813,N/A,N/A
2,AAP,58.9800,N/A,-0.633660,N/A,-49.970784,N/A,-13.869108,N/A,-13.519058,N/A,N/A
3,AAPL,175.3911,N/A,0.148946,N/A,10.409182,N/A,-4.535320,N/A,-0.264443,N/A,N/A
4,ABBV,153.5945,N/A,0.074121,N/A,-0.028126,N/A,13.471623,N/A,2.287223,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,0.105733,N/A,-2.497088,N/A,-8.336996,N/A,-2.686341,N/A,N/A
499,ZBH,118.4250,N/A,0.105733,N/A,-7.328428,N/A,-17.121560,N/A,-1.599498,N/A,N/A
500,ZBRA,230.5000,N/A,0.105733,N/A,-21.713139,N/A,-14.906971,N/A,-13.841438,N/A,N/A
501,ZION,34.4400,N/A,0.105733,N/A,6.658401,N/A,23.220028,N/A,-0.260643,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [24]:
time_periods = [
    '1 Year',
    '6 Months',
    '3 Months',
    '1 Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[change_col],
                                                                           hqm_dataframe.loc[row, change_col]) / 100

In [25]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,110.1300,N/A,-0.095828,0.107356,-19.334310,0.073559,-6.752049,0.290258,-7.344778,0.147117,N/A
1,AAL,13.2150,N/A,0.027537,0.163022,-8.037576,0.2167,-18.926376,0.053678,-12.829813,0.039761,N/A
2,AAP,58.9800,N/A,-0.633660,0.001988,-49.970784,0.005964,-13.869108,0.109344,-13.519058,0.027833,N/A
3,AAPL,175.3911,N/A,0.148946,0.823062,10.409182,0.67992,-4.535320,0.379722,-0.264443,0.588469,N/A
4,ABBV,153.5945,N/A,0.074121,0.184891,-0.028126,0.441352,13.471623,0.910537,2.287223,0.787276,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,0.105733,0.502982,-2.497088,0.355865,-8.336996,0.244533,-2.686341,0.395626,N/A
499,ZBH,118.4250,N/A,0.105733,0.502982,-7.328428,0.234592,-17.121560,0.065606,-1.599498,0.471173,N/A
500,ZBRA,230.5000,N/A,0.105733,0.502982,-21.713139,0.06163,-14.906971,0.093439,-13.841438,0.023857,N/A
501,ZION,34.4400,N/A,0.105733,0.502982,6.658401,0.600398,23.220028,0.982107,-0.260643,0.594433,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [26]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [27]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,110.1300,N/A,-0.095828,0.107356,-19.334310,0.073559,-6.752049,0.290258,-7.344778,0.147117,0.154573
1,AAL,13.2150,N/A,0.027537,0.163022,-8.037576,0.2167,-18.926376,0.053678,-12.829813,0.039761,0.11829
2,AAP,58.9800,N/A,-0.633660,0.001988,-49.970784,0.005964,-13.869108,0.109344,-13.519058,0.027833,0.036282
3,AAPL,175.3911,N/A,0.148946,0.823062,10.409182,0.67992,-4.535320,0.379722,-0.264443,0.588469,0.617793
4,ABBV,153.5945,N/A,0.074121,0.184891,-0.028126,0.441352,13.471623,0.910537,2.287223,0.787276,0.581014
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,125.3400,N/A,0.105733,0.502982,-2.497088,0.355865,-8.336996,0.244533,-2.686341,0.395626,0.374751
499,ZBH,118.4250,N/A,0.105733,0.502982,-7.328428,0.234592,-17.121560,0.065606,-1.599498,0.471173,0.318588
500,ZBRA,230.5000,N/A,0.105733,0.502982,-21.713139,0.06163,-14.906971,0.093439,-13.841438,0.023857,0.170477
501,ZION,34.4400,N/A,0.105733,0.502982,6.658401,0.600398,23.220028,0.982107,-0.260643,0.594433,0.66998


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [28]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [29]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [30]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,AKAM,108.4900,N/A,0.321598,0.926441,43.315723,0.988072,23.186100,0.980119,7.853665,0.974155,0.967197
1,CHTR,450.6550,N/A,0.341987,0.936382,27.921595,0.94831,37.068861,0.998012,7.278379,0.970179,0.963221
2,FICO,895.8425,N/A,1.056166,0.998012,28.550471,0.956262,13.326063,0.908549,5.966703,0.942346,0.951292
3,CEG,110.1975,N/A,0.289880,0.904573,44.910856,0.99006,22.090864,0.978131,3.852136,0.882704,0.938867
4,ACGL,80.8900,N/A,0.851127,0.994036,20.354115,0.870775,12.707264,0.902584,7.925287,0.976143,0.935885
5,FLT,270.9800,N/A,0.449384,0.964215,28.652141,0.95825,11.500643,0.884692,5.010666,0.926441,0.9334
6,CE,126.0000,N/A,0.362610,0.942346,21.297162,0.884692,16.756999,0.944334,6.770611,0.960239,0.932903
7,DLR,124.6200,N/A,0.192347,0.856859,30.120190,0.966203,20.583894,0.972167,4.015569,0.89662,0.922962
8,AJG,233.1800,N/A,0.363183,0.944334,27.463773,0.944334,9.751101,0.848907,5.553156,0.940358,0.919483
9,BR,184.3300,N/A,0.173825,0.840954,35.815196,0.982107,15.930084,0.934394,4.039311,0.898608,0.914016


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [31]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [32]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

200000.0

In [33]:
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Stock Price'])

hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,AKAM,108.4900,1843,0.321598,0.926441,43.315723,0.988072,23.186100,0.980119,7.853665,0.974155,0.967197
1,CHTR,450.6550,443,0.341987,0.936382,27.921595,0.94831,37.068861,0.998012,7.278379,0.970179,0.963221
2,FICO,895.8425,223,1.056166,0.998012,28.550471,0.956262,13.326063,0.908549,5.966703,0.942346,0.951292
3,CEG,110.1975,1814,0.289880,0.904573,44.910856,0.99006,22.090864,0.978131,3.852136,0.882704,0.938867
4,ACGL,80.8900,2472,0.851127,0.994036,20.354115,0.870775,12.707264,0.902584,7.925287,0.976143,0.935885
5,FLT,270.9800,738,0.449384,0.964215,28.652141,0.95825,11.500643,0.884692,5.010666,0.926441,0.9334
6,CE,126.0000,1587,0.362610,0.942346,21.297162,0.884692,16.756999,0.944334,6.770611,0.960239,0.932903
7,DLR,124.6200,1604,0.192347,0.856859,30.120190,0.966203,20.583894,0.972167,4.015569,0.89662,0.922962
8,AJG,233.1800,857,0.363183,0.944334,27.463773,0.944334,9.751101,0.848907,5.553156,0.940358,0.919483
9,BR,184.3300,1085,0.173825,0.840954,35.815196,0.982107,15.930084,0.934394,4.039311,0.898608,0.914016


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [34]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Formatting Our Excel Output

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [37]:
writer.close()